# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 6.3.2
- **Date :** 11/11/2023
- **Heure :** 13:24
- **Données :** `ready_Evaporation_Sunshine_WindGustDir_WindGustSpeed_Pressure9am_Pressure3pm_WindGustDirNum.csv`
- **Tri :** index chronologique
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `numpyfy`
- **Algorithme :** ROCKET

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../../data/processed/alex/ready_Evaporation_Sunshine_WindGustDir_WindGustSpeed_Pressure9am_Pressure3pm_WindGustDirNum.csv", index_col = 1).sort_index()
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Cloud9am,Cloud3pm,Temp9am,...,year,month,day,LocationNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum,NonMesNum
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-01,13.200000,24.083742,0.0,6.000000,19.000000,50.000000,24.000000,3.0,4.0,21.800000,...,2008,12,1,5,2.356194,2.356194,-32.919295,151.779535,0,3
2008-12-02,13.738447,32.200000,0.0,5.986227,19.000000,74.014791,59.000000,4.0,0.0,18.101224,...,2008,12,2,5,3.068698,0.785398,-32.919295,151.779535,0,3
2008-12-03,16.200000,32.200000,0.0,2.000000,2.000000,74.000000,26.000000,4.0,4.0,22.200000,...,2008,12,3,5,1.570796,5.497787,-32.919295,151.779535,0,3
2008-12-04,19.200000,24.000000,0.0,22.000000,22.000000,66.000000,61.000000,7.0,5.0,20.400000,...,2008,12,4,5,5.497787,5.497787,-32.919295,151.779535,0,3
2008-12-05,18.800000,30.200000,0.0,4.000000,10.614458,85.000000,57.328215,8.0,4.0,20.000000,...,2008,12,5,5,0.785398,4.021947,-32.919295,151.779535,0,3


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2929 entries, 2008-12-01 to 2017-06-24
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MinTemp        2929 non-null   float64
 1   MaxTemp        2929 non-null   float64
 2   Rainfall       2929 non-null   float64
 3   WindSpeed9am   2929 non-null   float64
 4   WindSpeed3pm   2929 non-null   float64
 5   Humidity9am    2929 non-null   float64
 6   Humidity3pm    2929 non-null   float64
 7   Cloud9am       2929 non-null   float64
 8   Cloud3pm       2929 non-null   float64
 9   Temp9am        2929 non-null   float64
 10  Temp3pm        2929 non-null   float64
 11  RainToday      2929 non-null   int64  
 12  RainTomorrow   2929 non-null   int64  
 13  year           2929 non-null   int64  
 14  month          2929 non-null   int64  
 15  day            2929 non-null   int64  
 16  LocationNum    2929 non-null   int64  
 17  WindDir9amNum  2929 non-null   float64
 18

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Conversion

In [7]:
# Importation de la fonction artisanale `numpyfy`

from numpyfy import numpyfy

In [8]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test)

## 4. Modélisation

In [9]:
# Instanciation d'un transformateur ROCKET

from sktime.transformations.panel.rocket import Rocket

rocket = Rocket()

In [10]:
# Récupération des paramètres initiaux du transformateur ROCKET

rocket.get_params()

{'n_jobs': 1, 'normalise': True, 'num_kernels': 10000, 'random_state': None}

In [11]:
# Ajustement du transformateur ROCKET au jeu d'entraînement

rocket.fit(X_train)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Rocket()

In [12]:
# Récupération des paramètres ajustés du transformateur ROCKET

rocket.get_fitted_params()

{}

In [13]:
# Application du transformateur ROCKET au jeu d'entraînement

X_train_transform = rocket.transform(X_train)

In [14]:
# Instanciation d'un modèle classificateur linéaire

from sklearn.linear_model import RidgeClassifierCV

clf_rocket_ridge = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10)) ## Pour ce paramétrage initial, nous suivons l'exemple fourni par `sktime`, qui fixe 10 alphas, de -3 à 3, à équidistance sur l'échelle logarithmique.

In [15]:
# Récupération des paramètres initiaux du modèle

clf_rocket_ridge.get_params()

{'alphas': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
        4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
        2.15443469e+02, 1.00000000e+03]),
 'class_weight': None,
 'cv': None,
 'fit_intercept': True,
 'scoring': None,
 'store_cv_values': False}

In [16]:
# Entraînement du modèle

clf_rocket_ridge.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]))

In [17]:
# Récupération des paramètres ajustés du modèle

clf_rocket_ridge.get_params()

{'alphas': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
        4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
        2.15443469e+02, 1.00000000e+03]),
 'class_weight': None,
 'cv': None,
 'fit_intercept': True,
 'scoring': None,
 'store_cv_values': False}

In [18]:
# Application du transformateur ROCKET au jeu de test

X_test_transform = rocket.transform(X_test)

In [19]:
# Réalisation des prédictions

y_pred = clf_rocket_ridge.predict(X_test_transform)

## 5. Évaluation

In [20]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0
Classe réelle,
0,376
1,112


In [21]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       376
           1       0.00      0.00      0.00       112

    accuracy                           0.77       488
   macro avg       0.39      0.50      0.44       488
weighted avg       0.59      0.77      0.67       488



/Users/omarchoa/anaconda3/envs/datascientest_project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/omarchoa/anaconda3/envs/datascientest_project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/omarchoa/anaconda3/envs/datascientest_project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh